<h1>Vehicle Routing Problem</h1>

In [1]:
from amplpy import AMPL, Constraint, DataFrame
from easygui import fileopenbox
from itertools import product
import pandas as pd

<h2>Structure of the Files </h2>

MAX_TIME  $T_{max}$

VEHICLE_SECTION

$k$ $U_{k}$ $C_{k}$

NODES_SECTION

$i$ $x_{i}$ $y_{i}$ $a_{i}$  $b_{i}$  $s_{i}$  $d_{i}^{B}$  $d_{i}^{L}$

In [2]:
file_path = fileopenbox()
file = open(file_path)

read_type = ""

T_max = 0
VCapacity = {}
VCost = {}
NLocation = {}
NTimeFrame = {}
NServiceTime = {}
NLinehaulDemand = {}
NBackhaulDemand = {}

for line in file.readlines():
    line = line.replace("\n","").strip()
    parts = line.split(" ")
    if parts[0] == "MAX_TIME":
        T_max = float(parts[1])
    elif parts[0] == "VEHICLE_SECTION":
        read_type = "VEHICLE_SECTION"
    elif parts[0] == "NODE_SECTION":
        read_type = "NODE_SECTION"
    elif read_type == "VEHICLE_SECTION":
        k, U_k, C_k = parts
        VCapacity[int(k)] = float(U_k)
        VCost[int(k)] = float(C_k) 
    elif read_type == "NODE_SECTION":
        i, x_i, y_i, a_i, b_i, s_i, d_i_B, d_i_L = parts
        NLocation[int(i)] = (float(x_i),float(y_i))
        NTimeFrame[int(i)] = (float(a_i),float(b_i))
        NServiceTime[int(i)] = float(s_i)
        NLinehaulDemand[int(i)] = float(d_i_L)
        NBackhaulDemand[int(i)] = float(d_i_B)
        

file.close()

In [3]:
def L2dist(a1, a2):
    return ((a1[0] - a2[0]) ** 2 + (a1[1]-a2[1]) ** 2) ** 0.5

ECost = {(i,j): L2dist(NLocation[i],NLocation[j]) for i,j in product(NLocation, NLocation)}
ETime = ECost.copy()
    

In [24]:
ampl = AMPL()

ampl.read("model.mod")

ampl.set["V"] = NServiceTime.keys()
ampl.set["warehouse"] = [0]
ampl.set["K"] = VCapacity.keys()

ampl.get_parameter("Tmax").set(T_max)
ampl.get_parameter("U").set_values(VCapacity)
ampl.get_parameter("CV").set_values(VCost)
ampl.get_parameter("t").set_values(ETime)
ampl.get_parameter("C").set_values(ECost)
ampl.get_parameter("a").set_values({k: NTimeFrame[k][0] for k in NTimeFrame})
ampl.get_parameter("b").set_values({k: NTimeFrame[k][1] for k in NTimeFrame})
ampl.get_parameter("s").set_values(NServiceTime)
ampl.get_parameter("dB").set_values(NBackhaulDemand)
ampl.get_parameter("dL").set_values(NLinehaulDemand)
ampl.get_parameter("M").set(10 ** 6)

ampl.option["solver"] = "gurobi"
ampl.solve()

Gurobi 10.0.2:Gurobi 10.0.2: optimal solution; objective 289.4427191
0 simplex iterations
 


In [25]:
ampl.get_variable("x").get_values().to_pandas()

x.val
index0 index1 index2       
0      0      1           0
       1      1           1
       2      1           0
1      0      1           0
       1      1           0
       2      1           1
2      0      1           1
       1      1           0
       2      1           0

In [26]:
ampl.get_variable("I").get_values().to_pandas()

,,I.val
index0,index1,
0,1,0
1,1,100
2,1,50
